In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

dataset_path = r"/content/drive/MyDrive/Bird Speciees Dataset"
if os.path.exists(dataset_path):
    print("✅ Dataset found!")
    print("📁 Bird species folders:", os.listdir(dataset_path))
else:
    print("❌ Dataset not found. Upload it manually.")

✅ Dataset found!
📁 Bird species folders: ['AMERICAN GOLDFINCH', 'BARN OWL', 'CARMINE BEE-EATER', 'DOWNY WOODPECKER', 'EMPEROR PENGUIN', 'FLAMINGO']


In [ ]:
!pip install tensorflow numpy matplotlib opencv-python

# CNN Model

In [ ]:
import os
import shutil
import random

# Paths to your dataset
dataset_path = "/content/drive/MyDrive/Bird Speciees Dataset"
train_path = "/content/drive/MyDrive/Bird Species Dataset/train"
val_path = "/content/drive/MyDrive/Bird Species Dataset/val"
test_path = "/content/drive/MyDrive/Bird Species Dataset/test"

# Create directories if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Define split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# List all class directories
classes = os.listdir(dataset_path)

for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    if not os.path.isdir(cls_path):
        continue

    # List all images in the class directory
    images = os.listdir(cls_path)
    random.shuffle(images)

    # Calculate split indices
    total_images = len(images)
    train_idx = int(total_images * train_ratio)
    val_idx = train_idx + int(total_images * val_ratio)

    # Split images into training, validation, and test sets
    train_images = images[:train_idx]
    val_images = images[train_idx:val_idx]
    test_images = images[val_idx:]

    # Create class subdirectories in train, val, and test
    os.makedirs(os.path.join(train_path, cls), exist_ok=True)
    os.makedirs(os.path.join(val_path, cls), exist_ok=True)
    os.makedirs(os.path.join(test_path, cls), exist_ok=True)

    # Move images to respective directories
    for img in train_images:
        shutil.copy(os.path.join(cls_path, img), os.path.join(train_path, cls, img))
    for img in val_images:
        shutil.copy(os.path.join(cls_path, img), os.path.join(val_path, cls, img))
    for img in test_images:
        shutil.copy(os.path.join(cls_path, img), os.path.join(test_path, cls, img))

print("Data split successfully!")

Data split successfully!


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/content/drive/MyDrive/Bird Species Dataset/train"  # Path to training dataset
val_dir = "/content/drive/MyDrive/Bird Species Dataset/val"  # Path to validation dataset
test_dir = "/content/drive/MyDrive/Bird Species Dataset/test"  # Path to test dataset

# Define ImageDataGenerator with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Define ImageDataGenerator for validation and test (only rescaling, no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load Training Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# Load Validation Data
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# Load Test Data
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False  # No shuffling for testing
)

print("Data Loaded Successfully!")

Found 565 images belonging to 6 classes.
Found 118 images belonging to 6 classes.
Found 128 images belonging to 6 classes.
Data Loaded Successfully!


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - accuracy: 0.1912 - loss: 2.5503 - val_accuracy: 0.5169 - val_loss: 1.4972
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 78s 4s/step - accuracy: 0.4641 - loss: 1.4570 - val_accuracy: 0.6780 - val_loss: 0.9034
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.6200 - loss: 1.0268 - val_accuracy: 0.7288 - val_loss: 0.7059
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.6712 - loss: 0.8608 - val_accuracy: 0.8559 - val_loss: 0.4467
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 83s 5s/step - accuracy: 0.7342 - loss: 0.7459 - val_accuracy: 0.8559 - val_loss: 0.3965
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.7394 - loss: 0.7187 - val_accuracy: 0.8390 - val_loss: 0.4098
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 83s 5s/step - accuracy: 0.7516 - loss: 0.6995 - val_accuracy: 0.8983 - val_loss: 0.2448
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 78s 4s/step - accuracy: 0.7628 - loss: 0.6755 - val_accuracy: 0.9153 - val_loss:

In [ ]:
# Evaluate Model on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Display Model Summary
model.summary()

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 975ms/step - accuracy: 0.8802 - loss: 0.4097
Test Accuracy: 0.8750
Test Loss: 0.3790


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_17 (Conv2D)                   │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,509,204 (127.83 MB)

 Trainable params: 11,169,734 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,339,470 (85.22 MB)

# Pre - Trained Model

In [ ]:
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
from tensorflow.keras.models import Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Use VGG16
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Use ResNet50 instead

for layer in base_model.layers:
    layer.trainable = False  # Freeze base model layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)  # 6 classes

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train Model
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    batch_size=32
)

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 445s 25s/step - accuracy: 0.2453 - loss: 1.8271 - val_accuracy: 0.7966 - val_loss: 1.2352
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 437s 25s/step - accuracy: 0.5700 - loss: 1.2619 - val_accuracy: 0.9068 - val_loss: 0.8385
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 436s 24s/step - accuracy: 0.7704 - loss: 0.9268 - val_accuracy: 0.8898 - val_loss: 0.6051
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 444s 25s/step - accuracy: 0.8415 - loss: 0.7287 - val_accuracy: 0.8898 - val_loss: 0.5188
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 444s 25s/step - accuracy: 0.8535 - loss: 0.6099 - val_accuracy: 0.9237 - val_loss: 0.4069
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 436s 24s/step - accuracy: 0.8835 - loss: 0.4919 - val_accuracy: 0.9068 - val_loss: 0.3536
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 443s 24s/step - accuracy: 0.8584 - loss: 0.4557 - val_accuracy: 0.9237 - val_loss: 0.2852
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 445s 25s/step - accuracy: 0.8963 - loss: 0.4106 - val_accuracy: 0.

In [ ]:
# Evaluate Model on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Display Model Summary
model.summary()

4/4 ━━━━━━━━━━━━━━━━━━━━ 86s 21s/step - accuracy: 0.9688 - loss: 0.1301
Test Accuracy: 0.9766
Test Loss: 0.1186


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 512)                 │         262,6

 Total params: 15,511,892 (59.17 MB)

 Trainable params: 265,734 (1.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 531,470 (2.03 MB)

In [ ]:
# Saving model in Google Drive
model_path = "/content/drive/MyDrive/bird_species_model.h5"
model.save(model_path)
print(f"Model saved successfully at: {model_path}")

Model saved successfully at: /content/drive/MyDrive/bird_species_model.h5


In [ ]:
import os

train_path = "/content/drive/MyDrive/Bird Species Dataset/train"

# Extract folder names (class labels)
class_labels = sorted(os.listdir(train_path))

# Print class labels to manually copy
print("Copy these labels and paste into Streamlit:\n")
print(class_labels)

Copy these labels and paste into Streamlit:

['AMERICAN GOLDFINCH', 'BARN OWL', 'CARMINE BEE-EATER', 'DOWNY WOODPECKER', 'EMPEROR PENGUIN', 'FLAMINGO']


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# ✅ Set the correct model path in Google Drive
MODEL_PATH = "/content/drive/MyDrive/bird_species_model.h5"

# ✅ Try loading the model
try:
    model = load_model(MODEL_PATH)
    print("✅ Model loaded successfully!")
except Exception as e:
    print("❌ ERROR: Model loading failed:", str(e))

✅ Model loaded successfully!


In [ ]:
model

<Functional name=functional_5, built=True>